In [2]:
import os, os.path as op
import glob
import argparse, sys, copy, time
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
# import seaborn as sns
import cProfile
import pstats
from datetime import datetime

%pdb off
%matplotlib notebook

Automatic pdb calling has been turned OFF


In [3]:
os.environ.get('PATH')

'/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'

#### Setup Structure

In [3]:
## you will need to edit this
path_AT  = op.join(op.expanduser('~'), 'ARIA-tools_git')

In [4]:
# get the git branch
os.chdir(path_AT)
gbranch = !git branch --show-current
gbranch = gbranch[0]
print ('On branch:', gbranch)

On branch: Profiling


In [5]:
# setup directory for aria outputs and profiles
track   = '004'

path_wd = op.join(op.expanduser('~'), f'Virtual_Tests_{track}') 
os.makedirs(path_wd, exist_ok=True)
os.chdir(path_wd)

path_profs = op.join(path_wd, 'Profiles')
os.makedirs(path_wd, exist_ok=True)

print ('Working directory:', path_wd)

Working directory: /home/jovyan/Virtual_Tests_004


In [6]:
path_dem  = op.join(path_wd, 'DEM', 'SRTM_3arcsec.dem')
bbox      = '36.75 37.225 -76.655 -75.928' # Hampton Roads
n_threads = 4

In [7]:
def clear_directory(path):
    """ Start from a clean directory """
    files = os.listdir(path)
    files = [op.join(path, f) for f in files]
    # keep the DEM as it saves some time
    if 'DEM' in files:
        files.remove('DEM')
        
    !rm -rf {' '.join(files)}
    return

## Single Frame

In [8]:
# 18 products between these dates for a single frame
st, en, nprod = 20170201, 20170501, 18 
# st, en, nprod = 20170101, 20180101, 80 ## longer runs


path_proc = op.join(path_wd, f'1Frame_{nprod}_{gbranch}')
os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [10]:
# start from a fresh directory
clear_directory(path_res)

print ('Ready to download ARIA to:', path_res)

Ready to download ARIA to: /home/jovyan/Virtual_Tests_004/1Frame_18_Profiling


### Download

In [11]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st} --en {en}  --b "{bbox}" -nt 4'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -nt 4
https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&processingLevel=GUNW_STD&output=JSON&relativeOrbit=004&bbox=-76.655,36.75,-75.928,37.225
Attempting to obtaining user/pass from .netrc
Using  4 threads for parallel downloads
 > attempting to download https://urs.earthdata.nasa.gov/profile
 > attempting to download https://urs.earthdata.nasa.gov/profile
 > attempting to download https://urs.earthdata.nasa.gov/profile
 > Reusing previous cookie jar.
 > Reusing previous cookie jar.
 > Reusing previous cookie jar.
 > 'Temporary' Redirect download @ Remote archive:
 > https://dlz0dhr6g6ukw.cloudfront.net/S1-GUNW-A-R-004-tops-20170323_20170227-230615-37654N_35777N-PP-22dc-v2_0_2.nc?userid=bbuzz&Expires=1650125133&Signature=th8IADKDwPGKnTV6PKofba8e35YHDDoz9mGVa~zVjxgyz6Nlvoc2rnWSfSOfHcO7N6bhm4VXx6pfWLXT4~5SeoYez5taqQ~Q8NRrzXFQyhUv5iaIlwspkWZKN2wzvcA7

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



 > 'Temporary' Redirect download @ Remote archive:
 > https://dlz0dhr6g6ukw.cloudfront.net/S1-GUNW-A-R-004-tops-20170227_20170203-230614-37655N_35778N-PP-cc96-v2_0_2.nc?userid=bbuzz&Expires=1650125136&Signature=wbHTEiioFKxlto4IFfCFgygXT8fa4HjKwZkSe~YaTEy-Mqh-HWnp~jTpeOa1~IDO28dcLw9NbcL2bc134wcEjxG1v5wu09Ydl2Zu0LP77KGK~F4uGcM~YSCVLe49sja5JvPXptDIWE3o3wIO7TB5t3VzUe7DYUW6~M~wGtLJMCw_&Key-Pair-Id=APKAINVNJF4BDB5SS5QQ
(2/4) Downloading https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20170227_20170203-230614-37655N_35778N-PP-cc96-v2_0_2.nc
 > Downloaded 37240832 of 58200507 bytes (63.99%))


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



 > 'Temporary' Redirect download @ Remote archive:
 > https://dlz0dhr6g6ukw.cloudfront.net/S1-GUNW-A-R-004-tops-20170416_20170323-230616-37654N_35778N-PP-b3c1-v2_0_2.nc?userid=bbuzz&Expires=1650125137&Signature=nJAHS89zCWAazRJ9v6Xr5hM1h-UasoNv~Z2YkQBVLK7JogNRB4Q2YMCGgGNhtlqrQGR4ET8sYr6zpswNCedt584OvhmX0HN9Y8oMMdqW7lqhEqjTUN0UgHP4onQD5wOPLKd6Q8vJ6RMi9w2~0NIwRWjWN3FPiyuDmW6muDmj~9E_&Key-Pair-Id=APKAINVNJF4BDB5SS5QQ
(5/5) Downloading https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-004-tops-20170416_20170323-230616-37654N_35778N-PP-b3c1-v2_0_2.nc
 > 'Temporary' Redirect download @ Remote archive:
 > Downloaded 44802048 of 66118621 bytes (67.76%)-R-004-tops-20170227_20170215-230614-37655N_35778N-PP-f19a-v2_0_2.nc?userid=bbuzz&Expires=1650125137&Signature=GCMpeBZpX~jcoAAWNb7Sm77zRUQQIMQw12~I3UPAH2Lu-FeB5uE6T9G-x-9m0vQcOwd1qL1Z5eXcfi1pfu3RfOeNnMHXB8gqYcQMU80XzXqn~qj3640r85oUiBG29kqIUgESBkfmt823kagnyY1Qu1IUPTHzKRxVsmasodqZGj4_&Key-Pair-Id=APKAINVNJF4BDB5SS5QQ > Downloaded 15818752 of 675

In [1]:
!which ariaDownload.py

In [11]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st} --en {en}  --b "{bbox}" -nt 4 -o url'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -nt 4 -o url
/bin/bash: ariaDownload.py: command not found


In [9]:
cmd = f'ariaExtract.py -f "{path_proc}/products/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

ariaExtract.py -f "./products/*.nc" --dem Download -nt 4
ARIA-tools Version: 1.1.0
*****************************************************************
*** Extract Product Function ***
*****************************************************************
Multi-core version
All (18) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (18) interferograms are spatially continuous.
No layers specified; only creating bounding box shapes
Thread count specified for gdal multiprocessing = 4
Applied cutline to produce 3 arc-sec SRTM DEM: ./DEM/SRTM_3arcsec.dem


### tsSetup1 - Physical

In [ ]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

### tsSetup1 - Virtual

In [ ]:
n     = 5 # run it nx

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/track004_bbox-77W36S-75E38N_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

## Multiple Frames

In [ ]:
# 21 products between these dates for multiple frames
st_tr, en_tr, nprod_tr = 20190101, 20190220, 21
# st_tr, en_tr, nprod_tr  = 20170101, 20170315, 83 

path_proc = op.join(path_wd, f'Track_{nprod_tr}_{gbranch}')
os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [ ]:
# start from a fresh directory
clear_directory(path_res)

print ('Ready to download ARIA to:', path_proc)

### Download

In [ ]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st_tr} --en {en_tr} -nt 4'
print (cmd)
!{cmd}

In [ ]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st_tr} --en {en_tr} -nt 4 -o url'
print (cmd)
!{cmd}

In [ ]:
cmd = f'ariaExtract.py -f "{path_proc}/products/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

### tsSetup+ - Physical

In [ ]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod_tr}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)

### tsSetup+ - Virtual

In [ ]:
n     = 5 # run it nx

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_AT}/bin/ariaTSsetup.py -f "{path_proc}/products/track004_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)